In [64]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

In [44]:
pc = Pinecone(api_key=os.getenv("PINECONE_DATABASE_SECRET_KEY"))
pc.create_index(
    name="rag-1", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

In [65]:
import json
data =json.load(open('reviews.json'))
data['reviews']

[{'professor': 'Dr. Alice Johnson',
  'subject': 'Mathematics',
  'stars': '5',
  'review': "Dr. Johnson's lectures are clear and engaging. She makes complex topics easy to understand."},
 {'professor': 'Dr. Bob Williams',
  'subject': 'Physics',
  'stars': '4',
  'review': 'Dr. Williams is knowledgeable and passionate, but his exams are tough.'},
 {'professor': 'Dr. Carol Smith',
  'subject': 'Chemistry',
  'stars': '3',
  'review': 'Dr. Smith is a decent professor, but her lectures can be a bit monotonous.'},
 {'professor': 'Dr. David Brown',
  'subject': 'Biology',
  'stars': '5',
  'review': 'Dr. Brown is an excellent professor who truly cares about his students.'},
 {'professor': 'Dr. Emily Davis',
  'subject': 'Computer Science',
  'stars': '4',
  'review': 'Dr. Davis is very knowledgeable, but her assignments can be quite challenging.'},
 {'professor': 'Dr. Frank Miller',
  'subject': 'Economics',
  'stars': '2',
  'review': "Dr. Miller's lectures are difficult to follow, and he

In [72]:
  
# processed_data = []
# client = OpenAI(api_key=os.getenv("OPENROUTER_API_KEY"))

# # Create embeddings for each review
# for review in data["reviews"]:
#     response = client.embeddings.create(
#         input=review['review'], model="openai/text-embedding-ada-002"
#     )
#     embedding = response.data[0].embedding
#     print("embeddings",embedding)
#     processed_data.append(
#         {
#             "values": embedding,
#             "id": review["professor"],
#             "metadata":{
#                 "review": review["review"],
#                 "subject": review["subject"],
#                 "stars": review["stars"],
#             }
#         }
#     )

HTTPError: 405 Client Error: Method Not Allowed for url: https://openrouter.ai/api/v1

In [48]:
index = pc.Index("rag-1")
upsert_response = index.upsert(
    vectors=processed_data,
    namespace="ns1",
)
print(f"Upserted count: {upsert_response['upserted_count']}")

PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Sun, 01 Sep 2024 10:32:50 GMT', 'Content-Type': 'application/json', 'Content-Length': '74', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '53', 'x-pinecone-request-id': '6845756645702318613', 'x-envoy-upstream-service-time': '54', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"No vectors provided for upsert request","details":[]}


In [49]:
print(index.describe_index_stats())


{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}
